<a href="https://colab.research.google.com/github/SERVIR/flood_mapping_intercomparison/blob/main/notebooks/Module_7_Object_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this module, we will calculate the intersection over union between flood maps. To do this, we will first need to export all of our flood maps to a common resolution.

# Step 1: Import packages

In [ ]:
import ee
import geemap
from google.colab import drive
import os
import glob
from osgeo import gdal
import numpy as np
import pandas as pd
import time

In [ ]:
ee.Authenticate()

ee.Initialize(project='servir-sco-assets')

# MODIFIABLE VARIABLE ALERT

In [ ]:
my_gee_folder = "users/mickymags/flood_intercomparison_chad_09_26_take2/"
my_Gdrive_folder = "/content/drive/MyDrive/Flood_Intercomparison/Case_Studies/confirmed_case_studies/cambodia_20241001/"
other_Gdrive_folder = "/content/drive/MyDrive/Flood_Intercomparison/Case_Studies/Flood_Intercomparison/"
flood_event_desc = 'chad_20240926'
time_id = '05280442'                # put in a string correlating to the current time which you are running the code -- will help with identifying exports

In [ ]:
aoi = ee.FeatureCollection(my_gee_folder + "aoi")
roi = aoi.geometry()
aoi_centroid = aoi.geometry().centroid()             # Get the center of the AOI
lon = aoi_centroid.coordinates().get(0).getInfo()    # Extract the longitude from the centroid
lat = aoi_centroid.coordinates().get(1).getInfo()    # Extract the latitude from the centroid

# Step 2: Resample GFM, MCDWD, and VFM products

In order to run object extraction statistics, we must have all products in a common projection. Thus, we will use gdal.Warp to resample these products to 30 meters

In [ ]:
drive.mount('/content/drive/')

In [ ]:
os.chdir(my_Gdrive_folder)

In [ ]:
pwd

In [ ]:
ls

## Step 2 Part 1: GFM

In [ ]:
os.chdir('../GFM')

In [ ]:
ls m*

In [ ]:
infile = glob.glob('merged*')[0]

In [ ]:
infile

In [ ]:
outfile = 'gfm_resampled' + flood_event_desc + '.tif'

In [ ]:
info = gdal.Info('merged_gfm_Cambodia_20241001.tif')
find = info.find('Data axis')
proj = info[find-8:find-3]
my_proj = 'EPSG:'+proj
print(my_proj)

In [ ]:
gdal.Warp(outfile, infile, dstSRS=my_proj, resampleAlg='mode', xRes=30, yRes=30)

In [ ]:
ls gfm_re*

# Step 2 Part 2: MCDWD

In [ ]:
os.chdir('../MCDWD')

In [ ]:
pwd

In [ ]:
mcdwd_infile = infile = glob.glob('merged*')[0]

In [ ]:
mcdwd_infile

In [ ]:
ls m*

In [ ]:
mcdwd_outfile = 'mcdwd_resampled_v3' + flood_event_desc + '.tif'

In [ ]:
gdal.Warp(mcdwd_outfile, mcdwd_infile, dstSRS=my_proj, resampleAlg='mode', xRes=30, yRes=30)

In [ ]:
ls mcdwd_re*

## Step 2 Part 3: VFM

In [ ]:
os.chdir('../VFM')

In [ ]:
pwd

In [ ]:
vfm_infile = infile = glob.glob('merged*')[0]

In [ ]:
vfm_outfile = 'vfm_resampled_takE5' + flood_event_desc + '.tif'

In [ ]:
vfm_infile

In [ ]:
gdal.Warp(vfm_outfile, vfm_infile, dstSRS=my_proj, resampleAlg='mode', xRes=30, yRes=30)

In [ ]:
ls v*

In [ ]:
pwd

# Step 3: Run Harmonization Module again

For GFM, MCDWD, and VFM, upload the resampled images to Google Earth Engine. Now we will rerun the harmonization module on each image

In [ ]:
mcdwd_resampled = ee.Image(my_gee_folder + 'mcdwd_resampled_mosaic')
gfm_resampled = ee.Image(my_gee_folder + 'gfm_resampled_mosaic')
vfm_resampled = ee.Image(my_gee_folder + 'vfm_resampled_mosaic')

## Step 3 Part 1: VFM Resampled Harmonization

In [ ]:
#############################################################################
#   Part 2: Create a function to reassign pixel values to our new schema    #
#############################################################################
# Define a function to transform VFM maps into common schema. This function takes an image as
#its input and outputs an image with the schema mentioned in Table 1
def vng_sch(image):

  ################ Get Water Pixels ####################

  # Raster Values of 15 indicate open water without water fraction retrieval
  # Raster Values of 99 indicate
  # Raster Values of 100 indicate Open Normal Water, from a river, lake, reservoir, or ocean
  # Raster Values of 101-200 idnicate water fractions of floodwater. We selected 50 as the percent threshold
  # to consider a pixel as water/nonwater. Thus, for the purposes of this study, we selected 150 as the threshold
  # above which pixels are considered as floodwater pixels.
  vfm_water = image.eq(15).Or(image.eq(99)).Or(image.eq(100)).Or(image.gte(150))

  ############## Get Nonwater Pixels ########################

  # Part 1 will get pixels that have floodwater but have less than 50% of the pixel covered in floodwater
  # Raster Values of 16 indicate clear-sky bare land
  # Raster Values of 17 indicate clear-sky vegetation
  # Raster Values of 20 indicate snow cover
  # Raster Values of 27 indicate river/lake covered in ice
  # Raster values of 38 indicate supra-snow ice water, mixed ice and water, or ice in melting status
  pt1 = image.gt(100).And(image.lt(150))
  pt2 = image.eq(16).Or(image.eq(17)).Or(image.eq(20)).Or(image.eq(27)).Or(image.eq(38))
  vfm_nonwater = pt1.Or(pt2)

  ############## Get Pixels where we do not have clear observations #############

  # Values of 1 indicate bad data pixels
  # Values of 30 indicate pixels where there is cloud cover
  # Values of 50 indicate pixels that have shadows from clouds or terrain.
  vfm_mask = image.eq(1).Or(image.eq(30)).Or(image.eq(50))

  # This section of the
  #ones = ee.Image(1)   # Dummy Image where every pixel has a value of 1
  #zeros = ee.Image(0)  # Dummy Image where every pixel has a value of 0
  #twos = ee.Image(2)   # Dummy Image where every pixel has a value of 2

  # This section of the code will replace the VFM raster values with our classification schema (see Table 1).
  vfm_mod = image.where(vfm_nonwater, ee.Image(0))   # Replace Nonwater Pixels as found by this code with a value of 0   # zeros
  vfm_v2 = vfm_mod.where(vfm_water, ee.Image(1))     # Replace Water Pixels as found by this code with a value of 1    # ones
  vfm_v3 = vfm_v2.where(vfm_mask, ee.Image(2))       # Replace Mask Pixels as found by this code with a value of 2    # twos

  return ee.Image(vfm_v3)

#################################################
#   Part 3: Apply our function from Part 2      #
#################################################
# Apply the dswx_sch function to our clipped dswx mosaic to the schema
# shown in table 1.
# Run the function on the mosaic
vfm_resampled_final = ee.Image(vng_sch(vfm_resampled)).clip(aoi)

## Step 3 Part 2: GFM Resampled Harmonization

In [ ]:
# We want to assign the pixels that were masked in the GFM image to have a value of 2.
def gfm_sch(image):
  twos = ee.Image(2)
  #wawa = image.eq(1)
  #nowawa = image.eq(0)
  masked = image.gte(2)

  #ones = ee.Image(1)
  #zeros = ee.Image(0)

  #gfm_new = img_2.where(wawa, ones)
  #gfm_fin = gfm_new.where(nowawa, zeros)
  gfm_fin = image.where(masked, twos)
  return gfm_fin

# Mosaic the individual GFM images and clip the mosaic to the area of interest
gfm_resampled_final = gfm_sch(gfm_resampled).clip(aoi)

## Step 3 Part 3: MCDWD Resampled Harmonization

In [ ]:
#####################################################
#  Part 1: Mosaic MCDWD Images, Clip them to aoi    #
#####################################################
#mcdwd_mos = ee.ImageCollection([mcdwd1, mcdwd2, mcdwd3]).mosaic().clip(aoi)

#############################################################################
#   Part 2: Create a function to reassign pixel values to our new schema    #
#############################################################################
def mcdwd_sch(image):

  #image = image.unmask()

  ################ Get Water Pixels ####################

  # Pixel Values of 1 indicate regular surface water according to the MCDWD algorithm
  # Pixel Values of 2 indicate a recurring flood according to the MCDWD algorithm
  # Pixel values of 3 indicate floodwater according to the MCDWD algorithm
  mcdwd_water = image.eq(1).Or(image.eq(2)).Or(image.eq(3))

  ############## Get Nonwater Values ########################
  # Pixel values of 0 indicate no water is present according to the MCDWD algorithm
  mcdwd_nonwater = image.eq(0)

  ############## Get Pixels where we do not have clear observations #############
  # Pixel values of 255 indicate insufficient data
  #mcdwd_mask = image.mask()               # Get the mask of the MCDWD map
  #mcdwd_maskdata = mcdwd_mask.eq(0)       # Find values where the MCDWD mask is equal to 0
  #mcdwd_insufficient = image.eq(255)            # Get the MCDWD values with insufficient data
  #mcdwd_unclear = mcdwd_maskdata.Or(mcdwd_insufficient)

  # Replace MCDWD pixel values with our classification schema (see Table 1)
  #mcdwd_mod = image.where(mcdwd_nonwater, ee.Image(0))
  #mcdwd_v2 = mcdwd_mod.where(mcdwd_water, ee.Image(1))
  #mcdwd_v3 = mcdwd_v2.where(mcdwd_unclear, ee.Image(2))
  #mcdwd_v3 = image.where(mcdwd_maskdata, ee.Image(2))

  mcdwd_mask = image.mask().clip(aoi)
  mcdwd_maskdata = mcdwd_mask.eq(0)
  mcdwd_insufficient = image.unmask().eq(255)
  mcdwd_unclear = mcdwd_maskdata.Or(mcdwd_insufficient)

  #mcdwd_v3 = image.unmask().where(mcdwd_maskdata, ee.Image(2))
  mcdwd_v0 = image.unmask()
  mcdwd_v1 = mcdwd_v0.where(mcdwd_nonwater, ee.Image(0))
  mcdwd_v2 = mcdwd_v1.where(mcdwd_water, ee.Image(1))
  mcdwd_v3 = mcdwd_v2.where(mcdwd_unclear, ee.Image(2))
  mcdwd_v4 = mcdwd_v3.clip(aoi)

  #return mcdwd_v3#.clip(aoi)
  return mcdwd_v4

#################################################
#   Part 3: Apply our function from Part 2      #
#################################################
# Apply the mcdwd_sch function to our clipped dswx mosaic to the schema
# shown in table 1.
mcdwd_resampled_final = mcdwd_sch(mcdwd_resampled).clip(aoi)

## Step 3 Part 4: Exporting to Google Earth Engine

In [ ]:
proj = mcdwd_resampled.projection().getInfo()['crs']

# Define a function that exports an Image to a Google Earth Engine Asset.
def exporter(img, asset_id):

  desc = 'Flood_Map_Export_'
  region_ = aoi.geometry()
  geemap.ee_export_image_to_asset(image = img,
                                  assetId = asset_id,
                                  description = desc,
                                  region = region_,
                                  crs = proj,
                                  scale = 30,
                                  maxPixels = 1e13)
  return 0

In [ ]:
# Export VFM
vfm_resampled_harmonized_aid = my_gee_folder + 'vfm_resampled_harmonized'
exporter(vfm_resampled_final, vfm_resampled_harmonized_aid)

# Export GFM
gfm_resampled_harmonized_aid = my_gee_folder + 'gfm_resampled_harmonized'
exporter(gfm_resampled_final, gfm_resampled_harmonized_aid)

# Export MCDWD
mcdwd_resampled_harmonized_aid = my_gee_folder + 'mcdwd_resampled_harmonized'
exporter(mcdwd_resampled_final, mcdwd_resampled_harmonized_aid)

# Step 4: Sampling

In [ ]:
dswxhls = ee.Image(my_gee_folder + 'dswxhls_harmonized_30')
dswxs1 = ee.Image(my_gee_folder + 'dswxs1_harmonized_30')
gfm = ee.Image(my_gee_folder + 'gfm_harmonized')
hydrafloods = ee.Image(my_gee_folder + 'hydrafloods_harmonized_30')
hydrosar = ee.Image(my_gee_folder + 'hydrosar_harmonized')
mcdwd = ee.Image(my_gee_folder + 'mcdwd_harmonized_30')
vfm = ee.Image(my_gee_folder + 'vfm_harmonized_30')
aoi = ee.FeatureCollection(my_gee_folder + "aoi")

In [ ]:
drive.mount('/content/drive/')

In [ ]:
os.chdir(other_Gdrive_folder)

In [ ]:
#mydesc = a string of the time you submitted
def iou(img1, img2, desc1, desc2, aoi, myproj, mydesc):   #num_pixels
  img1_renamed = img1.rename(desc1)
  img2_renamed = img2.rename(desc2)

  combo = img1_renamed.addBands(img2_renamed)

  # Add a random number to the end of the export string
  #random = np.random.randint(1e7)
  #randstr = str(random)

  # Tile aoi
  covering_grid = aoi.geometry().coveringGrid(myproj, 2e5)

  num_tiles = covering_grid.size().getInfo()

  for j in range(num_tiles):
    my_tile = ee.Feature(covering_grid.toList(num_tiles).get(j))
    my_tile_clipped = my_tile.geometry().intersection(aoi, 0.001)
    tile_id = str(j)
    tile_export_string = 'iou_'+desc1 + '_and_' + desc2 + '_' + mydesc + 'pt' + tile_id

    sample = combo.sample(
      region= my_tile_clipped,
      scale=30,
      projection = myproj,
      numPixels = 1e13#num_pixels
    )

    #export_string = 'iou_'+desc1 + '_and_' + desc2 + '_' + randstr

    print('sampling...')

    geemap.ee_export_vector_to_drive(
        collection=sample,
        description= tile_export_string,
        fileFormat='CSV',
        folder='Flood_Intercomparison',
        selectors = [desc1, desc2]
    )

  print('sleeping...')
  time.sleep(60)
  print('still sleeping...')
  time.sleep(120)

  # For each tile

  dataframes = []
  combined_dataframe = pd.DataFrame()

  for k in range(num_tiles):
    #Read file in
    another_id = str(k)
    tile_input_string = 'iou_'+desc1 + '_and_' + desc2 + '_'  + mydesc + 'pt' + another_id
    data = pd.read_csv(tile_input_string + '.csv')
    combined_dataframe = pd.concat([combined_dataframe, data])
    #dataframes.append(data)
    ######
  #data = pd.read_csv(export_string + '.csv')

  intersection = 0
  union = 0

  print('calculating iou...')

  #num_rows = int(num_pixels - 1)
  num_rows = int(np.floor(combined_dataframe.size / 2))
  print(num_rows)

  for j in range(num_rows):
    row = combined_dataframe.iloc[j]
    feat1 = row[desc1]
    feat2 = row[desc2]

    perc = j * 100 / num_rows

    if j % 500000 == 0:
      print('iou calculation is {0:0.1f} % complete'.format(perc))

    if feat1 == 2 or feat2 == 2:
      continue
    if feat1 == 1 and feat2 == 1:
      intersection += 1
    if feat1 == 1 or feat2 == 1:
      union += 1

  return intersection/union

In [ ]:
my_proj_code = gfm.projection().getInfo()['crs']

In [ ]:
pwd

In [ ]:
chad_gfm_hydrosar_iou = iou(gfm, hydrosar, 'gfm', 'hydrosar', aoi, my_proj_code, 1e7, '05281625')

In [ ]:
chad_gfm_hydrosar_iou

In [ ]:
chad_gfm_hydrafloods_iou = iou(gfm, hydrafloods, 'gfm', 'hydrafloods', aoi, my_proj_code, time_id)

In [ ]:
chad_gfm_mcdwd_iou = iou(gfm, mcdwd, 'gfm', 'mcdwd', aoi, my_proj_code, 1e13, time_id)